In [1]:
import numpy as np

In [2]:
def min_poly(sequence):
        # Standard Berlekamp-Massey algorithm over GF(2)
        N = len(sequence)
        s = sequence
        C = [1]         # connection polynomial
        B = [1]         # last "best" copy of C
        L = 0           # current LFSR length
        m = 1           # steps since last update of B

        for n in range(N):
            # compute discrepancy d = s[n] + sum_{i=1..L} C[i]*s[n-i]
            d = s[n]
            for i in range(1, L + 1):
                if i < len(C) and C[i]:
                    d ^= s[n - i]

            if d:  # non-zero discrepancy, adjust C
                T = C.copy()  # save current C

                # C = C + x^m * B
                shift = m
                if len(C) < len(B) + shift:
                    C.extend([0] * (len(B) + shift - len(C)))
                for j, coeff in enumerate(B):
                    C[j + shift] ^= coeff

                # update L, B, and reset m if needed
                if 2 * L <= n:
                    B = T
                    L = n + 1 - L
                    m = 1
                else:
                    m += 1
            else:
                m += 1

        return C

In [3]:
def find_minimal_polynomial(sequence):
        """
        Find the minimal polynomial of a binary sequence using the Berlekamp-Massey algorithm.

        Args:
            sequence: A binary sequence (list of 0s and 1s)

        Returns:
            The coefficients of the minimal polynomial as a list
        """
        # Length of the input sequence
        n = len(sequence)
        # Initialize connection polynomial c and previous best b as zero arrays
        c = np.zeros(n)
        b = np.zeros(n)
        # Set the first coefficient to 1 (polynomial starts with 1)
        c[0], b[0] = 1, 1
        # l: current LFSR length, m: last update index, i: current index
        l, m, i = 0, -1, 0
        # Convert input sequence to integers (in case it's not)
        int_data = [int(el) for el in sequence]
        # Iterate through the sequence
        while i < n:
            # Get the last l elements before position i, reversed
            v = int_data[(i - l):i]
            v = v[::-1]
            # Get the current connection polynomial coefficients (excluding the first)
            cc = c[1:l + 1]
            # Compute discrepancy d (should be 0 if current polynomial fits so far)
            d = (int_data[i] + np.dot(v, cc)) % 2
            if d == 1:
                # If discrepancy, save current c as temp
                temp = copy.copy(c)
                # Create a zero array for the update polynomial p
                p = np.zeros(n)
                # For each nonzero coefficient in b, update p at the shifted position
                for j in range(0, l):
                    if b[j] == 1:
                        p[j + i - m] = 1
                # Update connection polynomial c
                c = (c + p) % 2
                # If current LFSR is not long enough, update l, m, and b
                if l <= 0.5 * i:
                    l = i + 1 - l
                    m = i
                    b = temp
            # Move to next position
            i += 1
        # Return the minimal polynomial coefficients up to degree l
        return c[:l + 1].tolist()

In [4]:
import copy

# Generate 100 random binary sequences of length 20
num_sequences = 100
seq_length = 20
results = []

for _ in range(num_sequences):
    seq = np.random.randint(0, 2, seq_length).tolist()
    poly1 = min_poly(seq)
    poly2 = find_minimal_polynomial(seq)
    # Normalize both outputs to binary (0/1) and remove trailing zeros
    def normalize(poly):
        poly = [int(x) % 2 for x in poly]
        while len(poly) > 1 and poly[-1] == 0:
            poly.pop()
        return poly
    poly1_norm = normalize(poly1)
    poly2_norm = normalize(poly2)
    print(f"Sequence: {seq}\nPoly1: {poly1_norm}\nPoly2: {poly2_norm}\n")
    results.append(poly1_norm == poly2_norm)

# Print how many times the outputs matched
print(f"Outputs matched for {sum(results)} out of {num_sequences} sequences.")

Sequence: [0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1]
Poly1: [1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]
Poly2: [1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]

Sequence: [0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0]
Poly1: [1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1]
Poly2: [1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1]

Sequence: [1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0]
Poly1: [1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1]
Poly2: [1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1]

Sequence: [1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1]
Poly1: [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1]
Poly2: [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1]

Sequence: [0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1]
Poly1: [1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1]
Poly2: [1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1]

Sequence: [0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0]
Poly1: [1, 0, 1, 1, 0, 0, 0, 1, 0, 1]
Poly2: [1, 0, 1, 1, 0, 0, 0, 1, 0, 1]

Sequence: [0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,